In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from newspaper import Article
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bs
import datetime
import csv
import sys
import schedule
import os


def Crawling() :
    
    url='https://www.newsnow.co.uk/h/Industry+Sectors/Commodities/Grains%2C+Food+&+Fibre/Wheat?type=ln'

    options = webdriver.ChromeOptions()
    options.add_argument("headless")

    driver_path = '/usr/local/bin/chromedriver'
    driver=webdriver.Chrome(driver_path, options=options)
    driver.implicitly_wait(3)
    driver.get(url)

    res = requests.get(url)
    soup = bs(res.text, 'lxml')
    div = soup.select_one("div.rs-grid.rs-grid--skeleton.js-skeleton > main > div.rs-newsbox.js-newsbox.js-newsmain.js-central_ln_wrap > div").findAll('div', {"class" : "hl"})


    temp_link_list = []
    link_list = []
    name_list = []
    article_list = []
    date_list = []

    
    for hl in div :
        data = hl.find('a', {"class" : "hll"})
        time = hl.find('span', {'class' : "time"})
        
        date = time.text
        date = pd.to_datetime(date)
        date = date.strftime("%Y-%m-%d %H:%M:%S")
        date_list.append(date)

        temp_link_list.append(data.get('href'))
        name_list.append(data.text)


    for url in temp_link_list :
        res = requests.get(url)
        soup = bs(res.text, 'lxml')
        url = soup.select_one('script').get_text().split(" ")[13]
        urls = re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', str(url))
        urls = ''.join(urls)
        link_list.append(urls)


    for url in link_list :
        article = Article(url)
        try :
            article.download()
            article.parse()
            if len(article.text) > 30000 :
                article_list.append(article.summary)
            else :
                article_list.append(article.text)
        except :
            article_list.append('')


    df = pd.DataFrame({'link' : link_list,
                       'title': name_list,
                       'time': date_list,
                       'article': article_list
                      })
    

    return df


In [2]:
df = Crawling()

/var/folders/7n/qbcvswc55hzd4lnphhs0grb40000gn/T/ipykernel_66024/3299091911.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(driver_path, options=options)
/Users/anjaeu/opt/anaconda3/lib/python3.9/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [3]:
df.sort_index(axis=0, ascending=False, ignore_index=True, inplace=True)

In [4]:
df['link'] = df['link'].astype('string')
df['title'] = df['title'].astype('string')
df['time'] = df['time'].astype('string')
df['article'] = df['article'].astype('string')

In [5]:
df

,link,title,time,article
0,https://www.egyptindependent.com/global-wheat-...,Global wheat prices fell by 20% after the grai...,2022-07-24 11:51:00,Wheat prices fell on global stock exchanges by...
1,https://www.nasdaq.com/articles/grains-wheat-r...,GRAINS-Wheat rebounds on doubts over Ukrainian...,2022-07-25 03:53:00,"SINGAPORE, July 25 (Reuters) - Chicago wheat f..."
2,https://www.nasdaq.com/articles/grains-wheat-r...,GRAINS-Wheat rebounds as Russian attacks cast ...,2022-07-25 04:41:00,"By Naveen Thukral SINGAPORE, July 25 (Reuters..."
3,https://www.bnnbloomberg.ca/wheat-prices-jump-...,Wheat Prices Jump After Russian Missile Strike...,2022-07-25 04:58:00,(Bloomberg) -- Wheat prices jumped after Russi...
4,https://ukragroconsult.com/en/news/wheat-futur...,"Wheat futures close down nearly 50¢. Friday, J...",2022-07-25 05:13:00,"Wheat futures close down nearly 50¢. Friday, J..."
...,...,...,...,...
110,https://www.nasdaq.com/articles/grains-wheat-e...,"GRAINS-Wheat extends gains on strong demand, s...",2022-08-05 02:51:00,"SINGAPORE, Aug 5 (Reuters) - Chicago wheat fut..."
111,https://www.nasdaq.com/articles/grains-wheat-g...,GRAINS-Wheat gains more ground on strong deman...,2022-08-05 05:03:00,"By Naveen Thukral SINGAPORE, Aug 5 (Reuters) ..."
112,https://ukragroconsult.com/en/news/grains-end-...,"Grains end the day up. Thursday, August 4, 2022",2022-08-05 05:53:00,"Grains end the day up. Thursday, August 4, 202..."
113,https://ukragroconsult.com/en/news/us-2022-23-...,US 2022-23 spring wheat output seen at bumper ...,2022-08-05 05:53:00,US 2022-23 spring wheat output seen at bumper ...


In [6]:
df.to_csv('news_data_wheat.csv',index=False)